In [23]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean
from statsmodels.tsa.arima.model import ARIMA

In [22]:
pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 3.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean

In [3]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/25 08:32:14 WARN Utils: Your hostname, Ruths-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.192.116.5 instead (on interface en0)
24/04/25 08:32:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/25 08:32:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
DF1 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM renda") \
  .load()
DF1.show()

+----+--------------+-------------+----------+---------+-------------+------------+-------------+-----------------+
| Any|Codi_Districte|Nom_Districte|Codi_Barri|Nom_Barri|Seccio_Censal|Import_Euros|Import_Euros |__index_level_0__|
+----+--------------+-------------+----------+---------+-------------+------------+-------------+-----------------+
|2021|             1| Ciutat Vella|         1| el Raval|            1|     28484.0|         NULL|                0|
|2021|             1| Ciutat Vella|         1| el Raval|            2|     23611.0|         NULL|                1|
|2021|             1| Ciutat Vella|         1| el Raval|            3|     26607.0|         NULL|                2|
|2021|             1| Ciutat Vella|         1| el Raval|            4|     28688.0|         NULL|                3|
|2021|             1| Ciutat Vella|         1| el Raval|            5|     23769.0|         NULL|                4|
|2021|             1| Ciutat Vella|         1| el Raval|            6|  

In [10]:
DF2 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM rent_price") \
  .load()
DF2.show()

+----+---------+--------------+--------------------+--------------------+------+
|Year|Trimester|      District|       Neighbourhood|       Average _rent| Price|
+----+---------+--------------+--------------------+--------------------+------+
|2014|        1|  Ciutat Vella|            el Raval|average rent (eur...|589.55|
|2014|        1|  Ciutat Vella|      Gothic Quarter|average rent (eur...|712.79|
|2014|        1|  Ciutat Vella|      la Barceloneta|average rent (eur...|540.71|
|2014|        1|  Ciutat Vella|Sant Pere, Santa ...|average rent (eur...|673.44|
|2014|        1|      Eixample|          Fort Pienc|average rent (eur...|736.09|
|2014|        1|      Eixample|     Sagrada Familia|average rent (eur...|673.37|
|2014|        1|      Eixample|la Dreta de l'Eix...|average rent (eur...| 921.4|
|2014|        1|      Eixample|l'Antiga Esquerra...|average rent (eur...|827.87|
|2014|        1|      Eixample|la Nova Esquerra ...|average rent (eur...|716.13|
|2014|        1|      Eixamp

In [6]:
df=DF1.toPandas()
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros,Import_Euros,__index_level_0__
0,2021,1,Ciutat Vella,1,el Raval,1,28484.0,NaN,0
1,2021,1,Ciutat Vella,1,el Raval,2,23611.0,NaN,1
2,2021,1,Ciutat Vella,1,el Raval,3,26607.0,NaN,2
3,2021,1,Ciutat Vella,1,el Raval,4,28688.0,NaN,3
4,2021,1,Ciutat Vella,1,el Raval,5,23769.0,NaN,4
...,...,...,...,...,...,...,...,...,...
7471,2015,10,Sant Martí,73,la Verneda i la Pau,143,NaN,27409.0,1063
7472,2015,10,Sant Martí,65,el Clot,234,NaN,33389.0,1064
7473,2015,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,235,NaN,26240.0,1065
7474,2015,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,236,NaN,28535.0,1066


In [13]:
from unidecode import unidecode

def quitar_acentos_y_L(cadena):
    cadena_sin_acentos = unidecode(cadena)
    return cadena_sin_acentos.replace("L'", "")

In [11]:
df2=DF2.toPandas()
df2

,Year,Trimester,District,Neighbourhood,Average _rent,Price
0,2014,1,Ciutat Vella,el Raval,average rent (euro/month),589.55
1,2014,1,Ciutat Vella,Gothic Quarter,average rent (euro/month),712.79
2,2014,1,Ciutat Vella,la Barceloneta,average rent (euro/month),540.71
3,2014,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",average rent (euro/month),673.44
4,2014,1,Eixample,Fort Pienc,average rent (euro/month),736.09
...,...,...,...,...,...,...
4617,2022,2,Sant Marti,Diagonal Mar i el Front Maritim del Poblenou,average rent per surface (euro/m2),17.50
4618,2022,2,Sant Marti,el Besos i el Maresme,average rent per surface (euro/m2),12.20
4619,2022,2,Sant Marti,Provencals del Poblenou,average rent per surface (euro/m2),14.60
4620,2022,2,Sant Marti,Sant Marti de Provencals,average rent per surface (euro/m2),11.90


In [12]:
df2['District'].unique()

array(['Ciutat Vella', 'Eixample', 'Sants-Montjuic', 'Les Corts',
       'Sarria-Sant Gervasi', 'Gracia', 'Horta-Guinardo', 'Nou Barris',
       'Sant Andreu', 'Sant Marti'], dtype=object)

In [15]:
df['Nom_Districte'].unique()

array(['Ciutat Vella', 'Eixample', 'Sants-Montjuic', 'Les Corts',
       'Sarria-Sant Gervasi', 'Gracia', 'Horta-Guinardo', 'Nou Barris',
       'Sant Andreu', 'Sant Marti'], dtype=object)

In [16]:
df2['District'].unique() ==df['Nom_Districte'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [14]:
df['Nom_Districte']= df['Nom_Districte'].apply(quitar_acentos_y_L)
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros,Import_Euros,__index_level_0__
0,2021,1,Ciutat Vella,1,el Raval,1,28484.0,NaN,0
1,2021,1,Ciutat Vella,1,el Raval,2,23611.0,NaN,1
2,2021,1,Ciutat Vella,1,el Raval,3,26607.0,NaN,2
3,2021,1,Ciutat Vella,1,el Raval,4,28688.0,NaN,3
4,2021,1,Ciutat Vella,1,el Raval,5,23769.0,NaN,4
...,...,...,...,...,...,...,...,...,...
7471,2015,10,Sant Marti,73,la Verneda i la Pau,143,NaN,27409.0,1063
7472,2015,10,Sant Marti,65,el Clot,234,NaN,33389.0,1064
7473,2015,10,Sant Marti,69,Diagonal Mar i el Front Marítim del Poblenou,235,NaN,26240.0,1065
7474,2015,10,Sant Marti,69,Diagonal Mar i el Front Marítim del Poblenou,236,NaN,28535.0,1066


In [ ]:
DF1 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT Year, District, AVG(Price) AS Year_Price_month FROM rent_price WHERE "Average _Rent" = 'average rent (euro/month)' GROUP BY Year ,District''') \
  .load()
DF1.show()

In [ ]:
DF2 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT \"Any\", Nom_Districte, AVG(Import_Euros) AS Importe_Barrio FROM renda GROUP BY \"Any\",Nom_Districte") \
  .load()
DF2.show()

In [ ]:
DF3 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT Year, District, AVG(Price) AS Year_Price_m2 FROM rent_price WHERE "Average _Rent" = 'average rent per surface (euro/m2)' GROUP BY Year ,District''') \
  .load()
DF3.show()

In [ ]:
DF2 = DF2.withColumnRenamed("Nom_Districte", "Neighbourhood")
DF2 = DF2.withColumnRenamed("Any", "Year")

In [ ]:
DF_unidas = DF1.join(DF2, ["Neighbourhood", "Year"])
DF_unidas = DF_unidas.join(DF3, ["Neighbourhood", "Year"])
DF_unidas.show()

In [7]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

In [13]:
DF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", "SELECT * FROM rent_price") \
    .load()

In [14]:
DF.show()

+----+---------+--------------+--------------------+--------------------+------+-------------------+
|Year|Trimester|      District|       Neighbourhood|       Average _rent| Price|      Price_Escalat|
+----+---------+--------------+--------------------+--------------------+------+-------------------+
|2014|        1|  Ciutat Vella|            el Raval|average rent (eur...|589.55|0.23641140585517476|
|2014|        1|  Ciutat Vella|      Gothic Quarter|average rent (eur...|712.79|0.30156053413404094|
|2014|        1|  Ciutat Vella|      la Barceloneta|average rent (eur...|540.71|0.21059281266189483|
|2014|        1|  Ciutat Vella|Sant Pere, Santa ...|average rent (eur...|673.44|0.28075869870907033|
|2014|        1|      Eixample|          Fort Pienc|average rent (eur...|736.09| 0.3138777581595001|
|2014|        1|      Eixample|     Sagrada Familia|average rent (eur...|673.37| 0.2807216941733715|
|2014|        1|      Eixample|la Dreta de l'Eix...|average rent (eur...| 921.4| 0.41183933

In [27]:
data = DF.toPandas()
data

,Year,Trimester,District,Neighbourhood,Average _rent,Price,Price_Escalat
0,2014,1,Ciutat Vella,el Raval,average rent (euro/month),589.55,0.236411
1,2014,1,Ciutat Vella,Gothic Quarter,average rent (euro/month),712.79,0.301561
2,2014,1,Ciutat Vella,la Barceloneta,average rent (euro/month),540.71,0.210593
3,2014,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",average rent (euro/month),673.44,0.280759
4,2014,1,Eixample,Fort Pienc,average rent (euro/month),736.09,0.313878
...,...,...,...,...,...,...,...
4617,2022,2,Sant Marti,Diagonal Mar i el Front Maritim del Poblenou,average rent per surface (euro/m2),17.50,0.790287
4618,2022,2,Sant Marti,el Besos i el Maresme,average rent per surface (euro/m2),12.20,0.497792
4619,2022,2,Sant Marti,Provencals del Poblenou,average rent per surface (euro/m2),14.60,0.630243
4620,2022,2,Sant Marti,Sant Marti de Provencals,average rent per surface (euro/m2),11.90,0.481236


In [31]:
data[data['Neighbourhood']=='el Raval'][data['Average _rent']==]

,Year,Trimester,District,Neighbourhood,Average _rent,Price,Price_Escalat
0,2014,1,Ciutat Vella,el Raval,average rent (euro/month),589.55,0.236411
68,2014,2,Ciutat Vella,el Raval,average rent (euro/month),550.63,0.215837
138,2014,3,Ciutat Vella,el Raval,average rent (euro/month),576.45,0.229486
206,2014,4,Ciutat Vella,el Raval,average rent (euro/month),597.00,0.240350
273,2014,1,Ciutat Vella,el Raval,average rent per surface (euro/m2),10.76,0.418322
...,...,...,...,...,...,...,...
4284,2021,4,Ciutat Vella,el Raval,average rent per surface (euro/m2),13.10,0.547461
4352,2022,1,Ciutat Vella,el Raval,average rent (euro/month),917.00,0.409513
4419,2022,1,Ciutat Vella,el Raval,average rent per surface (euro/m2),15.50,0.679912
4486,2022,2,Ciutat Vella,el Raval,average rent (euro/month),872.00,0.385725


In [20]:
data = data['Price_Escalat']
data

0       0.236411
1       0.301561
2       0.210593
3       0.280759
4       0.313878
          ...   
4617    0.790287
4618    0.497792
4619    0.630243
4620    0.481236
4621    0.453642
Name: Price_Escalat, Length: 4622, dtype: float64

In [24]:
data
train = data.iloc[:-12]  # Datos de entrenamiento (primeros 4 años)
test = data.iloc[-12:]   # Datos de prueba (último año)

# Paso 5: Entrenar modelo
model = ARIMA(train, order=(5,1,0))  # Especifica el orden del modelo ARIMA
model_fit = model.fit()

# Paso 6: Validar modelo
predictions = model_fit.forecast(steps=len(test))  # Hacer predicciones para los datos de prueba

# Calcula el error (por ejemplo, MSE)
mse = ((predictions - test) ** 2).mean()
print("Mean Squared Error:", mse)

# Paso 7: Hacer predicciones para el siguiente año
next_year_predictions = model_fit.forecast(steps=12)

Mean Squared Error: 0.03670318737621794


In [25]:
predictions

4610    0.504106
4611    0.500111
4612    0.492843
4613    0.494655
4614    0.497096
4615    0.497505
4616    0.498338
4617    0.497594
4618    0.497381
4619    0.497334
4620    0.497279
4621    0.497432
Name: predicted_mean, dtype: float64

In [26]:
test

4610    0.475717
4611    0.503311
4612    0.591611
4613    0.569536
4614    0.674393
4615    0.966887
4616    0.757174
4617    0.790287
4618    0.497792
4619    0.630243
4620    0.481236
4621    0.453642
Name: Price_Escalat, dtype: float64